# Práctica #5 - Autómatas de pila

Milton Alejandro Cuervo Ramirez

Diego Alejandro Rendon Suaza

## **1. Introducción**

Esta es nuestra práctica de laboratorio centrada en los Autómatas de Pila (AP). Los AP son un modelo computacional fundamental en la teoría de lenguajes formales que extienden a los autómatas finitos con una memoria tipo pila. Esta capacidad de memoria les permite reconocer una clase más amplia de lenguajes: los Lenguajes Libres de Contexto (LLC). La práctica aborda la simulación de un AP para un lenguaje específico con una estructura que ejemplifica el poder de este modelo para manejar dependencias no locales.

## **2. Abstract**

This report documents the design and implementation of a computational simulator for a Non-deterministic Pushdown Automaton (NPDA). The main objective was to recognize and validate strings belonging to the formal language $L = { w^n d^m d^m w^n text{ with } n, m >= 0, w = (b+c)^* }$. The definition of the NPDA was formalized, including its states, alphabets, and transition function, and a simulation was developed in Python using a breadth-first search (BFS) approach to handle non-determinism. The work details the challenges encountered in defining precise transitions for this language, particularly the verification of the structure of the central block of 'd' characters, and the importance of a correct acceptance condition that considers the total consumption of the input string. The final implementation allows for the interactive evaluation of strings and provides a tool for the practical understanding of pushdown automata and context-free languages.

##**3.Cuerpo de la practica**

In [ ]:
import collections
import sys

# Definición del Autómata de Pila (APND)
# L = { w^n d^m d^m w^n | n, m >= 0, w = (b+c)^* }

# Estados:
# q0: Leyendo la primera parte w^n (empujando 'b's y 'c's a la pila)
# q1_after_w1: Después de leer la primera w^n. Espera el primer 'd' O es el caso m=0.
# q2_read_d1: Leyendo el primer bloque d^m.
# q3_between_d: Después del primer d^m. Espera el primer 'd' del segundo bloque.
# q4_read_d2: Leyendo el segundo bloque d^m.
# q5_before_w2: Después de leer ambos bloques d^m. Espera b/c para empezar a comparar.
# q6_match_w2: Leyendo la segunda w^n (sacando de la pila y comparando 'b's y 'c's)
# qA: Estado de aceptación (al final de la cadena y pila vacía)
states = {'q0', 'q1_after_w1', 'q2_read_d1', 'q3_between_d', 'q4_read_d2', 'q5_before_w2', 'q6_match_w2', 'qA'}

# Alfabeto de entrada: 'b', 'c', 'd'
input_alphabet = {'b', 'c', 'd'}

# Alfabeto de la pila: 'b', 'c', 'Z0' (símbolo inicial de la pila)
stack_alphabet = {'b', 'c', 'Z0'}
initial_stack_symbol = 'Z0'

# Estado inicial
initial_state = 'q0'

# Estado de aceptación
accepting_state = 'qA' # Usaremos aceptación por estado final Y pila vacía

# Función de Transición (Delta)
# delta[estado_actual][simbolo_entrada][simbolo_cima_pila] = [(siguiente_estado, accion_pila)]
# Acciones de pila: ('push', simbolo), ('pop', None), ('no_change', None)
# Simbolo_entrada puede ser None para transiciones epsilon

delta = {}

# Inicializar la estructura delta
for s in states:
    delta[s] = {}
    for i in list(input_alphabet) + [None]: # Incluir epsilon (None)
        delta[s][i] = {}
        for st in stack_alphabet:
            delta[s][i][st] = []

# Definir transiciones corregidas (basado en el análisis anterior):

# q0: Leyendo la primera w^n, empujar b/c
delta['q0']['b']['b'].append(('q0', ('push', 'b')))
delta['q0']['b']['c'].append(('q0', ('push', 'b')))
delta['q0']['b']['Z0'].append(('q0', ('push', 'b')))
delta['q0']['c']['b'].append(('q0', ('push', 'c')))
delta['q0']['c']['c'].append(('q0', ('push', 'c')))
delta['q0']['c']['Z0'].append(('q0', ('push', 'c')))
# Transición no determinista a q1_after_w1 (fin de la primera w^n)
delta['q0'][None]['b'].append(('q1_after_w1', ('no_change', None)))
delta['q0'][None]['c'].append(('q1_after_w1', ('no_change', None)))
delta['q0'][None]['Z0'].append(('q1_after_w1', ('no_change', None)))


# q1_after_w1: Después de leer la primera w^n. Espera el primer 'd' o pasa al caso m=0.
# Debe ver 'd' para empezar el primer bloque d^m
delta['q1_after_w1']['d']['b'].append(('q2_read_d1', ('no_change', None)))
delta['q1_after_w1']['d']['c'].append(('q2_read_d1', ('no_change', None)))
delta['q1_after_w1']['d']['Z0'].append(('q2_read_d1', ('no_change', None)))
# Caso m=0: Si no hay 'd's, salta los bloques d^m d^m y va directo a q5_before_w2
delta['q1_after_w1'][None]['b'].append(('q5_before_w2', ('no_change', None)))
delta['q1_after_w1'][None]['c'].append(('q5_before_w2', ('no_change', None)))
delta['q1_after_w1'][None]['Z0'].append(('q5_before_w2', ('no_change', None)))


# q2_read_d1: Leyendo el primer bloque d^m. Solo consumir 'd's.
delta['q2_read_d1']['d']['b'].append(('q2_read_d1', ('no_change', None)))
delta['q2_read_d1']['d']['c'].append(('q2_read_d1', ('no_change', None)))
delta['q2_read_d1']['d']['Z0'].append(('q2_read_d1', ('no_change', None)))
# Transición no determinista a q3_between_d (fin del primer bloque d^m)
delta['q2_read_d1'][None]['b'].append(('q3_between_d', ('no_change', None)))
delta['q2_read_d1'][None]['c'].append(('q3_between_d', ('no_change', None)))
delta['q2_read_d1'][None]['Z0'].append(('q3_between_d', ('no_change', None)))


# q3_between_d: Después del primer d^m. Espera el primer 'd' del segundo bloque.
# Debe ver 'd' para empezar el segundo bloque d^m. Eliminamos la transición epsilon directa a q5.
delta['q3_between_d']['d']['b'].append(('q4_read_d2', ('no_change', None)))
delta['q3_between_d']['d']['c'].append(('q4_read_d2', ('no_change', None)))
delta['q3_between_d']['d']['Z0'].append(('q4_read_d2', ('no_change', None)))
# No hay transición epsilon desde q3_between_d que salte q4_read_d2.


# q4_read_d2: Leyendo el segundo bloque d^m. Solo consumir 'd's.
delta['q4_read_d2']['d']['b'].append(('q4_read_d2', ('no_change', None)))
delta['q4_read_d2']['d']['c'].append(('q4_read_d2', ('no_change', None)))
delta['q4_read_d2']['d']['Z0'].append(('q4_read_d2', ('no_change', None)))
# Transición no determinista a q5_before_w2 (fin del segundo bloque d^m)
delta['q4_read_d2'][None]['b'].append(('q5_before_w2', ('no_change', None)))
delta['q4_read_d2'][None]['c'].append(('q5_before_w2', ('no_change', None)))
delta['q4_read_d2'][None]['Z0'].append(('q5_before_w2', ('no_change', None)))


# q5_before_w2: Después de leer ambos bloques d^m. Espera b/c para empezar a comparar.
# Debe ver 'b' o 'c' que coincida con la cima de la pila para empezar la comparación en q6
delta['q5_before_w2']['b']['b'].append(('q6_match_w2', ('pop', None)))
delta['q5_before_w2']['c']['c'].append(('q6_match_w2', ('pop', None)))
# Si la cadena termina aquí (input None) Y la pila solo tiene Z0, acepta (caso n=0, m=0 o n=0, m>0).
delta['q5_before_w2'][None]['Z0'].append(('qA', ('pop', None)))


# q6_match_w2: Leyendo la segunda w^n, comparando con la pila.
delta['q6_match_w2']['b']['b'].append(('q6_match_w2', ('pop', None)))
delta['q6_match_w2']['c']['c'].append(('q6_match_w2', ('pop', None)))
# Si la cadena termina aquí (input None) Y la pila solo tiene Z0, acepta (caso n>0, m>=0).
delta['q6_match_w2'][None]['Z0'].append(('qA', ('pop', None)))


# qA: Estado de aceptación - No tiene transiciones salientes relevantes en la simulación

# --- Función de Simulación ---

def simulate_pda(input_string, debug=True):
    """
    Simula el APND para determinar si input_string es aceptada.
    Utiliza búsqueda en anchura (BFS) para explorar caminos.
    Aceptación por estado final (qA) y pila vacía.
    Incluye mensajes de depuración si debug=True.
    """
    # La cola almacenará tuplas: (estado_actual, cadena_restante, pila_actual)
    q = collections.deque([(initial_state, input_string, [initial_stack_symbol])])

    # Usamos un conjunto para rastrear configuraciones visitadas y evitar bucles infinitos
    # Una configuración es (estado, indice_input, tupla_pila)
    visited = set()

    if debug:
        print(f"\n--- Simulación para '{input_string}' ---")
        print(f"Cola inicial: {[(initial_state, input_string, [initial_stack_symbol])]}")
        print("---")

    step_count = 0
    while q:
        step_count += 1
        current_state, remaining_input, current_stack = q.popleft()
        input_index = len(input_string) - len(remaining_input)
        current_stack_tuple = tuple(current_stack) # Usar tupla para set

        if debug:
             print(f"Paso {step_count}: Explorando (Estado='{current_state}', Input='{remaining_input}', Pila={current_stack})")

        # Si esta configuración ya fue visitada, saltarla
        if (current_state, input_index, current_stack_tuple) in visited:
            if debug:
                print("    -> Configuración ya visitada. Saltando.")
            continue

        # Marcar la configuración como visitada
        visited.add((current_state, input_index, current_stack_tuple))

        # Obtener el símbolo de la cima de la pila
        stack_top = current_stack[-1] if current_stack else None
        if stack_top is None: # Esto solo debería pasar si la pila se vació antes de pop(Z0)
             if debug:
                 print("    -> Pila vacía inesperadamente. Ruta muere.")
             continue # Ruta inválida

        # --- Explorar transiciones ---
        possible_moves = []

        # 1. Transiciones con símbolo de entrada (si input no está vacío)
        current_char = remaining_input[0] if remaining_input else None
        if current_char is not None and current_char in delta.get(current_state, {}):
             if stack_top in delta[current_state][current_char]:
                 possible_moves.extend([(current_char, move) for move in delta[current_state][current_char][stack_top]])


        # 2. Transiciones epsilon
        if None in delta.get(current_state, {}):
            if stack_top in delta[current_state][None]:
                 possible_moves.extend([(None, move) for move in delta[current_state][None][stack_top]])

        if debug:
            if not possible_moves:
                print("    -> No hay transiciones posibles. Ruta muere.")
            else:
                print(f"    -> Transiciones posibles: {possible_moves}")


        for consumed_char, (next_state, stack_action) in possible_moves:
            next_stack = list(current_stack) # Copiar la pila para la nueva rama
            action_type, action_sym = stack_action

            if action_type == 'push':
                next_stack.append(action_sym)
            elif action_type == 'pop':
                if not next_stack:
                    if debug: print("    -> Intentó POP en pila vacía. Ruta inválida.")
                    continue # Ruta inválida: intentar pop de pila conceptualmente vacía
                next_stack.pop()
            elif action_type == 'no_change':
                pass # No se modifica la pila

            next_remaining_input = remaining_input[1:] if consumed_char is not None else remaining_input # Consumir el caracter si no fue epsilon

            if debug:
                 print(f"    -> Aplicando transición: ({current_state}, {consumed_char if consumed_char is not None else 'ε'}, {stack_top}) -> ({next_state}, {stack_action})")
                 print(f"       Nueva Config: (Estado='{next_state}', Input='{next_remaining_input}', Pila={next_stack})")


            # --- CORRECCIÓN CRUCIAL: Verificar condición de aceptación ---
            # Aceptar solo si:
            # 1. Se alcanza el estado de aceptación (qA)
            # 2. La pila está vacía (se sacó Z0)
            # 3. Y la CADENA DE ENTRADA HA SIDO COMPLETAMENTE CONSUMIDA
            if next_state == accepting_state and not next_stack and not next_remaining_input:
                 if debug:
                     print(f"--- Cadena '{input_string}' ACEPTADA ---")
                 return True # Cadena aceptada

            # Enqueue la nueva configuración
            # Solo encolamos si no hemos llegado al estado de aceptación final todavía
            # (la condición de aceptación ya maneja el retorno)
            if next_state != accepting_state:
                 q.append((next_state, next_remaining_input, next_stack))
            else:
                 if debug:
                     print("    -> Alcanzó estado de aceptación pero Input/Pila no cumplen condición final. Ruta muere.")


    # Si la cola se vacía y no hemos llegado a un estado de aceptación válido, la cadena es rechazada
    if debug:
         print(f"--- Simulación terminada. Cadena '{input_string}' RECHAZADA ---")
    return False

# --- Interfaz de usuario simple ---

if __name__ == "__main__":
    print("Simulador de Autómata de Pila para L = { w^n d^m d^m w^n | w=(b+c)^* }")
    print("Ingrese cadenas para probar (o 'salir' para terminar).")
    print("Los pasos de depuración se muestran por defecto.")


    debug_mode = True # Empezar con depuración activada

    while True:
        user_input = input("\nIngrese una cadena: ")
        if user_input.lower() == 'salir':
            break

        # Validar que la cadena solo contenga b, c, d
        if not all(char in input_alphabet for char in user_input):
            print("Error: La cadena solo puede contener 'b', 'c' y 'd'.")
            continue

        if simulate_pda(user_input, debug=debug_mode):
            print("=> ACEPTADA")
        else:
            print("=> RECHAZADA")

        # Preguntar si quiere depuración en la siguiente ronda (opcional, simplificado)
        # debug_choice = input("Mostrar pasos de depuración para la siguiente cadena? (s/n): ").lower()
        # debug_mode = (debug_choice == 's' or debug_choice == 'd')

## **5. Conclusiones**

Mediante la realización de esta práctica, se logró implementar exitosamente la simulación computacional de un Autómata de Pila No Determinista para reconocer cadenas que se ajustan a la estructura del lenguaje $L = \{ w^n d^m d^m w^n \}$.

Se consolidó la comprensión sobre:
- La definición formal de un Autómata de Pila y sus componentes (estados, alfabetos, función de transición, pila).
- La importancia de la pila como memoria auxiliar para reconocer Lenguajes Libres de Contexto.
- La naturaleza no determinista de algunos lenguajes y cómo simular la exploración de múltiples caminos de cómputo (utilizando BFS).
- La formulación de la función de transición para guiar al autómata a través de las diferentes partes de una cadena y realizar operaciones de apilamiento y desapilamiento en el momento adecuado.
- La importancia de una correcta condición de aceptación, verificando no solo el estado final y la pila, sino también que toda la entrada haya sido procesada.

Aunque el lenguaje presenta una estructura ($d^m d^m$) que excede las capacidades de verificación completa de un AP estándar en combinación con la estructura $w^n w^n$, la simulación implementada modela un APND que reconoce la correspondencia de $w^n$ alrededor de un bloque central de 'd's, que es la parte típicamente manejable por un AP en lenguajes de este tipo. La práctica permitió aplicar los conceptos teóricos y enfrentar los desafíos prácticos de la simulación de autómatas no deterministas.